In [317]:
%matplotlib inline
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
from __future__ import print_function
from numpy import median, diff

In [324]:
class SongFile:
    def __init__(self, name, extension):
        self.name = name
        self.extension = extension
        self.music_file = 'StepMania/Songs/StepMania 5/{0}/{0}.{1}'.format(name, extension)
        self.stepfile = 'StepMania/Songs/StepMania 5/{0}/{0}.ssc'.format(name)
        self.data, self.sample_rate = librosa.load(self.music_file, sr=22050*8)
        self.tempo, self.beat_frames = librosa.beat.beat_track(y=self.data, sr=self.sample_rate)
        self.beat_times = librosa.frames_to_time(self.beat_frames, sr=self.sample_rate)

In [327]:
A = SongFile('A', 'mp3')
B = SongFile('B', 'ogg')
C = SongFile('C', 'ogg')

In [407]:
def get_beats(beat_times, beat_frames):
    changes = []
    changes_time = []
    for i in range(len(beat_frames) - 1):
        changes.append(beat_frames[i + 1] - beat_frames[i])
        changes_time.append(beat_times[i + 1] - beat_times[i])

    sorted_changes = sorted(changes)
    median = sorted_changes[int(len(changes) / 2)]

    changes_counted = [abs(change - median) < 2 for change in changes]
    time_changes_counted = list(itertools.compress(changes_time, changes_counted))
    average = sum(time_changes_counted) / len(time_changes_counted)
    
    time_differences = []
    earliest_proper_beat = 1
    for i in range(1, len(beat_times) - 1):
        if changes_counted[i] & changes_counted[i - 1]:
            earliest_proper_beat = i
            break
            
    last_proper_beat = len(beat_times) -2
    for i in range(1, len(beat_times) - 1):
        if changes_counted[len(beat_times) - i - 1] & changes_counted[len(beat_times) - i - 2]:
            last_proper_beat = len(beat_times) - i - 1
            break
    
    time_differences = []
    buffer = 5
    for i in range(20):
        start_beat = earliest_proper_beat + buffer * i
        if changes_counted[start_beat] & changes_counted[start_beat - 1]:
            for j in range(20):
                end_beat = last_proper_beat - buffer * j
                if changes_counted[end_beat] & changes_counted[end_beat - 1]:
                    time_differences.append(beat_times[end_beat] - beat_times[start_beat])
        
    # get num beats, round, and make new average
    new_averages = [time_difference / round(time_difference / average) for time_difference in time_differences]
    #print (new_averages)
    new_averages.sort()
    num_averages = len(new_averages)
    #new_average = sum(new_averages[5:num_averages - 5]) / (num_averages - 10)
    new_average = new_averages[int(num_averages/2)]
    return [(0, 60./new_average)]
    
def get_time_string(times):
    time_strings = []
    for time in times:
        if time[0] != 0:
            time_strings.append(',')
        time_strings.append('{:}={:.3f}'.format(time[0], time[1]))
    return ''.join(time_strings)

def test_get_beats(beat_times, beat_frames):
    a = get_beats(beat_times, beat_frames)
    a.append((500, 1))
    time = beat_times[0]
    beat_times_mock = [time]
    for i in range(len(a) - 1):
        for j in range(a[i+1][0] - a[i][0]):
            time += 60/a[i][1]
            beat_times_mock.append(time)

    beat_times_pd = pd.DataFrame(beat_times, columns=['VALUE'])
    beat_times_pd['REAL'] = True
    beat_times_pd
    beat_times_mock_pd = pd.DataFrame(beat_times, columns=['VALUE'])
    beat_times_mock_pd['REAL'] = False
    both = beat_times_pd.append(beat_times_mock_pd)
    both['BEAT']=both.index
    linestyles = ["--", "-"]*390
    fig, (ax) = plt.subplots(1,1, figsize=(20,7))
    graph = sb.pointplot(x='BEAT', y='VALUE', hue='REAL', linestyles=linestyles, data=both, ax=ax)
    
#test_get_beats(beat_times)

In [408]:
test_1 = get_beats(A.beat_times, A.beat_frames)[0][1] - 181.685
test_2 = get_beats(B.beat_times, B.beat_frames)[0][1] - 140
test_3 = get_beats(C.beat_times, C.beat_frames)[0][1] - 180

print (test_1)
print (test_2)
print (test_3)
print (sum([abs(result) for result in [test_1, test_2, test_3]]))

-0.0105715210007
-0.00987584650113
-0.0369012253992
0.057348592901


In [385]:
print (60./181.685)
print (60./140)
print (60./180)

0.33024190219335664
0.42857142857142855
0.3333333333333333


In [309]:
def write_song_header(output_stepfile, info_map):
    keys = ['VERSION', 'TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH']
    header_info = {
        'VERSION': 0.82,
        'TITLE': info_map['song_name'],
        'MUSIC': '{0}.mp3'.format(info_map['song_name']),
        'OFFSET': -0.090,
        'SAMPLESTART': info_map['sample_start'],
        'SAMPLELENGTH': info_map['sample_length']
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, info_map):
    print("//---------------dance-single - ----------------", file=output_stepfile)
    keys = ['NOTEDATA', 'CHARTNAME', 'STEPSTYPE', 'DIFFICULTY', 'METER', 'RADARVALUES', 'BPMS']
    step_info = {
        'NOTEDATA': '',
        'CHARTNAME': 'Kommisar',
        'STEPSTYPE': 'dance-single',
        'DIFFICULTY': 'Beginner',
        'METER': 1,
        'RADARVALUES': '0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0,0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0',
        'BPMS': info_map['bpm']
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(step_info[key])), file=output_stepfile)
        
def write_notes(output_stepfile, info_map):
    print ("#NOTES:", file=output_stepfile)
    
    for i in range(80):
        print ("0101\n0001\n0101\n0001\n,", file=output_stepfile)
    print ("0000;", file=output_stepfile)

In [329]:
def get_info_map(song):
    info_map = {}
    info_map['song_name'] = song.name
    info_map['song_format'] = song.extension
    times = get_beats(song.beat_times, song.beat_frames)
    bpm_string = get_time_string(times)
    info_map['bpm'] = bpm_string
    tempo = 60./times[0][1]
    info_map['sample_start'] = 16 * tempo
    info_map['sample_length'] = 32 * tempo
    return info_map

In [330]:
def step_song(song):
    output_stepfile=open(song.stepfile, 'w')
    info_map = get_info_map(song)
    write_song_header(output_stepfile, info_map)
    write_step_header(output_stepfile, info_map)
    write_notes(output_stepfile, info_map)
    output_stepfile.close()

In [333]:
step_song(A)
step_song(B)
step_song(C)

In [380]:
[1, 2, 3][0:3]

[1, 2, 3]